In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import *
from sklearn.metrics import classification_report,plot_confusion_matrix,confusion_matrix,ConfusionMatrixDisplay,roc_curve,RocCurveDisplay,precision_recall_curve,PrecisionRecallDisplay

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from sklearn.utils import shuffle
import cv2
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import EfficientNetB3
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

__<h2>class for data preprocessing and network</h2>__

In [ ]:
class main_class():
    def __init__(self,path_add):
        self.path_add=path_add

    def data_frame(self):
        images_link = list(self.path_add.glob(r'**/*.*'))
        images_name = [x.parents[0].stem for x in images_link]
        images_labels=preprocessing.LabelEncoder().fit_transform(images_name)

        df=pd.DataFrame(list(zip(images_link,images_name,images_labels)),columns=['Link','Name','label'])
        return df

    def img_show(self,image_links,plot_size,):
        df=image_links.sample(frac=1)
        fig = plt.figure(1, figsize=(30, 30))
        grid = ImageGrid(fig,121, nrows_ncols=(plot_size, 2), axes_pad=0.10)
        i = 0
        for category_id, category in enumerate(df.Name.unique()):
            for filepath in df[df['Name'] == category]['Link'].values[:2]:
                ax = grid[i]
                #img = image.load_img(filepath,(150,150))
                #img=image.img_to_array(img)
                #ax.imshow(img/255)


                image = Image.open(filepath).convert("L")
                arr = np.asarray(image)
                ax.imshow(arr, cmap='gray', vmin=0, vmax=255)
                ax.axis('off')

                if i % 2 == 2 - 1:
                    ax.text(600, 200, category, verticalalignment='center',fontsize=20,color = 'red')


                i+=1

        plt.show()

    def EfficientNet_call(self,df,EfficientNet,imsize,epoch,batch):
        image_data=[]
        for i in df.Link:
            img_pre=cv2.imread(str(i))
            img_pre=cv2.resize(img_pre, (imsize,imsize))
            image_data.append(img_pre)

        x=np.array(image_data)
        y=to_categorical(df.label)
        x_data, y_data=shuffle(x,y,random_state=42)
        xtrain,xtest,ytrain,ytest=model_selection.train_test_split(x_data, y_data, test_size=0.1, random_state=42)

        efficient = EfficientNet(weights='imagenet',include_top = False, input_shape=(imsize, imsize, 3))

        model = efficient.output
        model = tf.keras.layers.GlobalAveragePooling2D()(model)
        model = tf.keras.layers.Dropout(0.5)(model)
        model = tf.keras.layers.Dense(4, kernel_regularizer=tf.keras.regularizers.l2(0.01),activation
             ='softmax')(model)
        model = tf.keras.models.Model(inputs = efficient.input , outputs = model)
        model.compile(loss = 'squared_hinge',
                      optimizer = "adam",
                      metrics=["accuracy"]
                      )
        model.summary()



        tensorboard = TensorBoard(log_dir="logs")
        checkpoint = ModelCheckpoint("efficient.h5",
                                     monitor='val_accuracy',
                                     mode = "auto",
                                     verbose = 1,
                                     save_best_only =True)
        reduce_LR = ReduceLROnPlateau(monitor="val_accuracy",
                                      factor = 0.3,
                                      patience = 2,
                                      min_delta=0.001,
                                      mode = "auto",
                                      verbose = 1)
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
        history = model.fit(xtrain, ytrain,
                  validation_data =(xtest,ytest),
                  epochs = epoch,
                  verbose =1,
                  batch_size = batch,
                  callbacks = [tensorboard, checkpoint, reduce_LR,es])

        return model,xtrain,xtest,ytrain,ytest,history


__<h2>for data frame</h2>__

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp /content/drive/MyDrive/brain_tumor/archive.zip /content/


In [ ]:
!unzip /content/archive.zip

In [ ]:
!cp -r /content/Testing /content/dataset/Testing/
!cp -r /content/Training /content/dataset/Training/

In [ ]:
dataset_path = Path("/content/", "dataset")
data_info=main_class(dataset_path)
df=data_info.data_frame()

In [ ]:
print(df)

__<h2>random image plot</h2>__

In [ ]:
from PIL import Image
name_image=np.random.choice(np.unique(df.Name))
data_info.img_show(df[df.Name==name_image],2)

__<h2>model fit</h2>__

In [ ]:
main_dt=data_info.EfficientNet_call(df,EfficientNetB3,224,20,32)

__<h1>predict test image plot</h1>

In [ ]:
result_pre = np.argmax(main_dt[0].predict(main_dt[2]), axis=1)
result_acc = np.argmax(main_dt[4], axis=1)

plt.figure(figsize=(20, 20))
for i in range(16):
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(main_dt[2][i])
    plt.title('glioma' if result_pre[i]==0
                else 'meningioma' if result_pre[i]==1
                else 'notumor' if result_pre[i]==2
                else 'pituitary',fontsize=20)
    plt.suptitle('Test Image plot and labels',fontsize=30)
    plt.axis("off")

plt.show()

__<h2>confusion_matrix plot</h2>__

In [ ]:
con_max = confusion_matrix(result_acc,result_pre)
cm = confusion_matrix(result_acc,result_pre)
cmp = ConfusionMatrixDisplay(cm,display_labels = ['glioma', 'meningioma','notumor', 'pituitary'])
fig, ax = plt.subplots(figsize=(20,120))
cmp.plot(ax=ax, xticks_rotation=90)

__<h2>classification_report</h2>__

In [ ]:
print(classification_report(result_acc,result_pre,target_names=['glioma', 'meningioma','notumor', 'pituitary']))

__<h2>model accuracy,loss data frame with epoch </h2>__

In [ ]:
df_acc=pd.DataFrame()
df_acc['Loss']=main_dt[5].history['loss']
df_acc['Value Loss']=main_dt[5].history['val_loss']
df_acc['Accuracy']=main_dt[5].history['accuracy']
df_acc['Value Accuracy']=main_dt[5].history['val_accuracy']
df_acc

__<h2>Top accuracy</h2>__

In [ ]:
df_acc.sort_values('Loss',ascending=False).head(1).plot(kind='bar',figsize = (12,8))

__<h2>Loss,Accuracy polt</h2>__

In [ ]:
# summarize history for accuracy
plt.plot(main_dt[5].history['accuracy'])
plt.plot(main_dt[5].history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(main_dt[5].history['loss'])
plt.plot(main_dt[5].history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()